# Ray Tracing: Is our analytical approach going to work?

# Third question: What is the time difference between the time arrivals at different stations from the same array?

To answer this question, we are going to compute the time difference for the direct P-wave, the direct S-wave, the PPP and 3SH waves reflected off the mid-slab discontinuity, and the PPSSSand the 5SH waves reflected off the Moho.

Load Python modules.

In [1]:
import numpy
from math import sqrt

Load functions from my own ray-tracing related modules.

In [2]:
from computeAngle import computeAngle3, computeAngle5
from computeTravelTime import computeTravelTime3, computeTravelTime5
from misc import latLon2xy

Get P- and S-wave velocities in the continental crust.

In [3]:
from data import VpCC, VsCC

Get latitudes, longitudes and names of the stations. Get names of the arrays.

In [4]:
from data import lat_r, lon_r, names_r, nr, names_a, na

Get latitudes, longitudes and depths of the tremor sources.

In [5]:
from data import lat_s, lon_s, d_s, ns

Get latitude and longitude of the center of the (tremor sources) grid.

In [6]:
from data import lat0, lon0

Compute the distances from the tremor sources to the center of the (tremor sources) grid.

In [7]:
(x_s, y_s) = latLon2xy(lat_s, lon_s, lat0, lon0)

Initializations.

In [8]:
dtP_S = numpy.zeros((na, ns))
dtP_PPP = numpy.zeros((na, ns))
dtS_PPP = numpy.zeros((na, ns))
dtP_3SH = numpy.zeros((na, ns))
dtS_3SH = numpy.zeros((na, ns))
dtP_PPSSS = numpy.zeros((na, ns))
dtS_PPSSS = numpy.zeros((na, ns))
dtP_5SH = numpy.zeros((na, ns))
dtS_5SH = numpy.zeros((na, ns))

Open output file.

In [9]:
output = open('TimeLagOnGrid.txt', 'w')

Loop on arrays.

In [10]:
for i in range(0, na):
    output.write('Array {}:\n'.format(names_a[i]))
    output.write('---------\n')
    # Compute the distances from the stations to the center of the (tremor sources) grid
    (x_r, y_r) = latLon2xy(lat_r[i], lon_r[i], lat0, lon0)
    # Initializations
    tP = numpy.zeros((nr[i], ns))
    tS = numpy.zeros((nr[i], ns))
    tPPP = numpy.zeros((nr[i], ns))
    t3SH = numpy.zeros((nr[i], ns))
    tPPSSS = numpy.zeros((nr[i], ns))
    t5SH = numpy.zeros((nr[i], ns))
    # Loop on source position
    for j in range(0, ns):
        # Loop on stations
        for k in range(0, nr[i]):
            # Ray P
            tP[k, j] = sqrt((x_r[k] - x_s[j]) ** 2.0 + (y_r[k] - y_s[j]) ** 2.0 + d_s[j] ** 2.0) / VpCC
            # Ray S
            tS[k, j] = sqrt((x_r[k] - x_s[j]) ** 2.0 + (y_r[k] - y_s[j]) ** 2.0 + d_s[j] ** 2.0) / VsCC
            # Ray PPP
            angle = computeAngle3(x_s[j], y_s[j], d_s[j], x_r[k], y_r[k], 'P', 'P', 'P')
            tPPP[k, j] = computeTravelTime3(angle, x_s[j], y_s[j], d_s[j], x_r[k], y_r[k], 'P', 'P', 'P')
            # Ray 3SH
            angle = computeAngle3(x_s[j], y_s[j], d_s[j], x_r[k], y_r[k], 'S', 'S', 'S')
            t3SH[k, j] = computeTravelTime3(angle, x_s[j], y_s[j], d_s[j], x_r[k], y_r[k], 'S', 'S', 'S')
            # Ray PPSSS
            angle = computeAngle5(x_s[j], y_s[j], d_s[j], x_r[k], y_r[k], 'P', 'P', 'S', 'S', 'S')
            tPPSSS[k, j] = computeTravelTime5(angle, x_s[j], y_s[j], d_s[j], x_r[k], y_r[k], 'P', 'P', 'S', 'S', 'S')
            # Ray 5SH
            angle = computeAngle5(x_s[j], y_s[j], d_s[j], x_r[k], y_r[k], 'S', 'S', 'S', 'S', 'S')
            t5SH[k, j] = computeTravelTime5(angle, x_s[j], y_s[j], d_s[j], x_r[k], y_r[k], 'S', 'S', 'S', 'S', 'S')
        for k in range(0, nr[i] - 1):
            for l in range(k + 1, nr[i]):
                dtS_3SH[i, j] = dtS_3SH[i, j] + abs((t3SH[k, j] - tS[k, j]) - (t3SH[l, j] - tS[l, j]))
                dtS_5SH[i, j] = dtS_5SH[i, j] + abs((t5SH[k, j] - tS[k, j]) - (t5SH[l, j] - tS[l, j]))
                dtS_PPSSS[i, j] = dtS_PPSSS[i, j] + abs((tPPSSS[k, j] - tS[k, j]) - (tPPSSS[l, j] - tS[l, j]))
                dtP_PPP[i, j] = dtP_PPP[i, j] + abs((tPPP[k, j] - tP[k, j]) - (tPPP[l, j] - tP[l, j]))
                dtP_S[i, j] = dtP_S[i, j] + abs((tS[k, j] - tP[k, j]) - (tS[l, j] - tP[l, j]))
                dtS_PPP[i, j] = dtS_PPP[i, j] + abs((tS[k, j] - tPPP[k, j]) - (tS[l, j] - tPPP[l, j]))
                dtP_3SH[i, j] = dtP_3SH[i, j] + abs((t3SH[k, j] - tP[k, j]) - (t3SH[l, j] - tP[l, j]))
                dtP_PPSSS[i, j] = dtP_PPSSS[i, j] + abs((tPPSSS[k, j] - tP[k, j]) - (tPPSSS[l, j] - tP[l, j]))
                dtP_5SH[i, j] = dtP_5SH[i, j] + abs((t5SH[k, j] - tP[k, j]) - (t5SH[l, j] - tP[l, j]))
        dtS_3SH[i, j] = dtS_3SH[i, j] * 2.0 / ((nr[i] - 1) * nr[i])
        dtS_5SH[i, j] = dtS_5SH[i, j] * 2.0 / ((nr[i] - 1) * nr[i])
        dtS_PPSSS[i, j] = dtS_PPSSS[i, j] * 2.0 / ((nr[i] - 1) * nr[i])
        dtP_PPP[i, j] = dtP_PPP[i, j] * 2.0 / ((nr[i] - 1) * nr[i])
        dtP_S[i, j] = dtP_S[i, j] * 2.0 / ((nr[i] - 1) * nr[i])
        dtS_PPP[i, j] = dtS_PPP[i, j] * 2.0 / ((nr[i] - 1) * nr[i])
        dtP_3SH[i, j] = dtP_3SH[i, j] * 2.0 / ((nr[i] - 1) * nr[i])
        dtP_PPSSS[i, j] = dtP_PPSSS[i, j] * 2.0 / ((nr[i] - 1) * nr[i])
        dtP_5SH[i, j] = dtP_5SH[i, j] * 2.0 / ((nr[i] - 1) * nr[i])

    output.write('Autocorrelation - Horizontal component\n')
    output.write('\n')
    output.write('Time difference between S-wave and SH-wave (mid-slab)\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtS_3SH[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtS_3SH[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtS_3SH[i, :])))
    output.write('\n')
    output.write('Time difference between S-wave and SH-wave (Moho)\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtS_5SH[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtS_5SH[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtS_5SH[i, :])))
    output.write('\n')
    output.write('Time difference between S-wave and PPSSS-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtS_PPSSS[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtS_PPSSS[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtS_PPSSS[i, :])))
    output.write('\n')

    output.write('Autocorrelation - Vertical component\n')
    output.write('\n')
    output.write('Time difference between P-wave and PPP-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtP_PPP[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtP_PPP[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtP_PPP[i, :])))
    output.write('\n')

    output.write('Cross correlation - Horizontal and vertical component\n')
    output.write('\n')
    output.write('Time difference between S-wave and P-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtP_S[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtP_S[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtP_S[i, :])))
    output.write('\n')   
    output.write('Time difference between S-wave and PPP-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtS_PPP[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtS_PPP[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtS_PPP[i, :])))
    output.write('\n')
    output.write('Time difference between SH-wave (mid-slab) and P-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtP_3SH[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtP_3SH[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtP_3SH[i, :])))
    output.write('\n')
    output.write('Time difference between PPSSS-wave and P-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtP_PPSSS[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtP_PPSSS[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtP_PPSSS[i, :])))
    output.write('\n')   
    output.write('Time difference between SH-wave (Moho) and P-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(dtP_5SH[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(dtP_5SH[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(dtP_5SH[i, :])))
    output.write('\n')

Average on all arrays.

In [11]:
output.write('All arrays:\n')
output.write('---------  \n')

output.write('Autocorrelation - Horizontal component\n')
output.write('\n')
output.write('Time difference between S-wave and SH-wave (mid-slab)\n')
output.write('Min value: {} s\n'.format(numpy.min(dtS_3SH)))
output.write('Max value: {} s\n'.format(numpy.max(dtS_3SH)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtS_3SH)))
output.write('\n')
output.write('Time difference between S-wave and SH-wave (Moho)\n')
output.write('Min value: {} s\n'.format(numpy.min(dtS_5SH)))
output.write('Max value: {} s\n'.format(numpy.max(dtS_5SH)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtS_5SH)))
output.write('\n')
output.write('Time difference between S-wave and PPSSS-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(dtS_PPSSS)))
output.write('Max value: {} s\n'.format(numpy.max(dtS_PPSSS)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtS_PPSSS)))
output.write('\n')

output.write('Autocorrelation - Vertical component\n')
output.write('\n')
output.write('Time difference between P-wave and PPP-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(dtP_PPP)))
output.write('Max value: {} s\n'.format(numpy.max(dtP_PPP)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtP_PPP)))
output.write('\n')

output.write('Cross correlation - Horizontal and vertical component\n')
output.write('\n')
output.write('Time difference between S-wave and P-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(dtP_S)))
output.write('Max value: {} s\n'.format(numpy.max(dtP_S)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtP_S)))
output.write('\n')   
output.write('Time difference between S-wave and PPP-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(dtS_PPP)))
output.write('Max value: {} s\n'.format(numpy.max(dtS_PPP)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtS_PPP)))
output.write('\n')
output.write('Time difference between SH-wave (mid-slab) and P-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(dtP_3SH)))
output.write('Max value: {} s\n'.format(numpy.max(dtP_3SH)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtP_3SH)))
output.write('\n')
output.write('Time difference between PPSSS-wave and P-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(dtP_PPSSS)))
output.write('Max value: {} s\n'.format(numpy.max(dtP_PPSSS)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtP_PPSSS)))
output.write('\n')   
output.write('Time difference between SH-wave (Moho) and P-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(dtP_5SH)))
output.write('Max value: {} s\n'.format(numpy.max(dtP_5SH)))
output.write('Mean value: {} s\n'.format(numpy.mean(dtP_5SH)))
output.write('\n')

1

Close output file.

In [12]:
output.close()